In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/Users/brandon/Desktop/dummy_encoder.csv')

In [149]:

print(invalid_columns)



[{'name': 'cat_a', 'values': ['one', 'two', 'three'], 'type': 'object', 'count': 3}, {'name': 'cat_b', 'values': ['a', 'b'], 'type': 'object', 'count': 2}, {'name': 'outcome', 'values': [True, False], 'type': 'bool', 'count': 2}]


In [152]:
from sklearn.preprocessing import OneHotEncoder

valid_data_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

valid = df.select_dtypes(include=valid_data_types)
invalid = df.drop(columns=valid.columns)

invalid_columns = []
for column in invalid.columns:
    uniq = list(invalid[column].unique())

    invalid_columns.append(
    {
        'name': column,
        'values': uniq,
        'type': str(invalid[column].dtype),
        'count': len(invalid[column].unique())
    })

column_map = []
for col_data in invalid_columns:
    
    #Ensure Boolean Values are Preserved
    if col_data['type'] == 'bool':
        invalid[col_data['name']] = invalid[col_data['name']].astype('int')
    
    #Ensure Nonboolean Binary Variables are kept as a single column
    elif col_data['count'] == 2:
        mapping = {}
        for index, val in enumerate(col_data['values']):
            mapping[val] = index
        column_map.append({
            'column': col_data['name'],
            'map': mapping
        })
        invalid[col_data['name']] = invalid[col_data['name']].map(mapping).astype('int') 
    
    #Dummy encode
    elif col_data['count'] > 2:
        
        enc = OneHotEncoder(handle_unknown='ignore')
        
        #Need 2D vector to do the fit
        vector = invalid[col_data['name']].values.reshape(-1,1)
        enc.fit(vector)
        
        trans = enc.transform(vector).toarray()
        
        temp_df = pd.DataFrame(trans, columns=enc.categories_).add_prefix(col_data['name'] + '_').astype('int')
        
        #Adjust original columns
        invalid = pd.concat([temp_df,invalid], axis=1)
        invalid = invalid.drop(col_data['name'], axis=1)
        

pd.concat([valid, invalid], axis=1).to_json()
    

 
        

'{"va_a":{"0":1,"1":2,"2":3,"3":4,"4":5,"5":6,"6":7,"7":8,"8":9,"9":10},"va_b":{"0":0.145,"1":0.942,"2":0.36,"3":0.678,"4":0.464,"5":0.694,"6":0.308,"7":0.978,"8":0.009,"9":0.607},"va_c":{"0":0.817,"1":0.759,"2":0.767,"3":0.579,"4":0.488,"5":0.587,"6":0.516,"7":0.018,"8":0.035,"9":0.996},"(\'cat_a_one\',)":{"0":1,"1":0,"2":0,"3":1,"4":0,"5":0,"6":1,"7":0,"8":0,"9":1},"(\'cat_a_three\',)":{"0":0,"1":0,"2":1,"3":0,"4":0,"5":1,"6":0,"7":0,"8":1,"9":0},"(\'cat_a_two\',)":{"0":0,"1":1,"2":0,"3":0,"4":1,"5":0,"6":0,"7":1,"8":0,"9":0},"cat_b":{"0":0,"1":1,"2":0,"3":1,"4":0,"5":1,"6":0,"7":1,"8":0,"9":1},"outcome":{"0":1,"1":0,"2":1,"3":0,"4":1,"5":0,"6":1,"7":0,"8":1,"9":0}}'

In [102]:

enc.fit(invalid['cat_a'].values.reshape(-1,1))
array = enc.transform(invalid['cat_a'].values.reshape(-1,1)).toarray()
pd.DataFrame(array, columns=enc.categories_, ).add_prefix('test_')


,test_one,test_three,test_two
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
5,0.0,1.0,0.0
6,1.0,0.0,0.0
7,0.0,0.0,1.0
8,0.0,1.0,0.0
9,1.0,0.0,0.0


In [11]:

enc = OneHotEncoder(handle_unknown='ignore')





ModuleNotFoundError: No module named 'sklearn'